In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm, linear_model

from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import StandardScaler,PolynomialFeatures, LabelEncoder,FunctionTransformer
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from scipy import stats
%matplotlib inline
titanic = pd.read_csv('train.csv')
testdf = pd.read_csv('test.csv')
import pandas as pd
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

/opt/conda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
testdf.drop('PassengerId', axis=1, inplace=True)
titanic.drop('PassengerId', axis=1, inplace=True)

In [3]:

titanic['FamilyCount']= (titanic['SibSp'])+(titanic['Parch'])
titanic['IsFemale'] = (titanic.Sex=='female').astype(int)
titanic['IsMale'] = (titanic.Sex=='male').astype(int)
testdf['FamilyCount']= (testdf['SibSp'])+(testdf['Parch'])
testdf['IsFemale'] = (testdf.Sex=='female').astype(int)
testdf['IsMale'] = (testdf.Sex=='male').astype(int)
titanic['Cabin'] =titanic['Cabin'].fillna(value='?')
titanic['Embarked'] =titanic['Embarked'].fillna(value='S')
titanic['Age'] =titanic['Age'].fillna(value='?')
testdf['Cabin'] =testdf['Cabin'].fillna(value='?')
testdf['Embarked'] =testdf['Embarked'].fillna(value='S')
testdf['Age'] =testdf['Age'].fillna(value='?')
testdf.loc[152,'Fare']=13.6755

In [4]:
titanic['IsAlone']= (titanic['FamilyCount']==0).astype(int)
testdf['IsAlone']= (testdf['FamilyCount']==0).astype(int)

In [5]:
def titlepull(title_string):
    title_list = title_string.split()
    for possible_title in title_list:
        if possible_title[-1] == '.':
            return possible_title
def cabinpull(cabin_string):
    if cabin_string[0]=='?':
        return '?'
    else:
        return cabin_string[0]
    

In [6]:
def ticketpull(ticketstring):
    if ticketstring == 'LINE':
        return 9
    string = ticketstring.split(' ')[-1]
    return string[0]
    

In [7]:
titanic['Deck']=titanic.Cabin.apply(cabinpull)
titanic['Title']= titanic.Name.apply(titlepull)
testdf['Deck']=testdf.Cabin.apply(cabinpull)
testdf['Title']= testdf.Name.apply(titlepull)
titanic['Ticketnum']= titanic.Ticket.apply(ticketpull)
testdf['Ticketnum']=testdf.Ticket.apply(ticketpull)


In [8]:
decks_list = ['?','C','B','D','E','F','A','G','T']

In [9]:
name_mappings = {
    'Dona.':'Royalty',
    'Countess.':'Royalty',
    'Sir.':'Royalty',
    'Jonkheer.':'Royalty',
    'Don.':'Royalty',
    'Lady.':'Royalty',
    'Ms.':'Miss.',
    'Col.':'Officer',
    'Capt.':'Officer',
    'Major.':'Officer',
    'Col.':'Officer',
    'Mlle.':'Miss.',
    'Mme.':'Mrs.',
    'Mrs.':'Mrs.',
    'Miss.':'Miss',
    'Mr.':'Mr.',
    'Master.':'Master.',
    'Dr.':'Dr.',
    'Rev.':'Rev.'
    
    
}

In [10]:
titanic['Title']= titanic['Title'].map(name_mappings)
testdf['Title']= testdf['Title'].map(name_mappings)

In [11]:
validtitles = list(set(name_mappings.values()))
titanic.FamilyCount = titanic['FamilyCount'].astype("category", categories=range(11), ordered=False)


In [12]:
testdf.FamilyCount = testdf['FamilyCount'].astype("category", categories=range(11), ordered=False)
titanic.Title = titanic['Title'].astype("category", categories=validtitles, ordered=False)
testdf.Title = testdf['Title'].astype("category", categories=validtitles, ordered=False)
titanic.Deck = titanic['Deck'].astype("category", categories=decks_list, ordered=False)
testdf.Deck = testdf['Deck'].astype("category", categories=decks_list, ordered=False)
kmg = KMeans()

def numeric_features(df):
    return df.select_dtypes(exclude= ['object','category'])
kmg.fit(numeric_features(titanic.drop('Survived', axis=1)))
titanic['cluster'] =  kmg.predict(numeric_features(titanic.drop('Survived', axis=1)))
testdf['cluster'] = kmg.predict(numeric_features(testdf))

numeric_features_tf = FunctionTransformer(numeric_features, validate=False)

In [13]:
testdf.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsMale', 'IsAlone',
       'Deck', 'Title', 'Ticketnum', 'cluster'],
      dtype='object')

In [14]:
age_transformers = [('numfeat' ,numeric_features_tf)]

In [15]:
fu_age_impute = FeatureUnion(age_transformers)

In [16]:
knr = KNeighborsRegressor(n_jobs=-1)
lr = LinearRegression()
ss = StandardScaler()

In [17]:
age_impute_pipe_lr =  Pipeline(
    [
        ('fu',fu_age_impute),
        ('ss',ss),
        ('lr',lr)
    ]

)
age_impute_pipe_kn =  Pipeline(
    [
        ('fu_age',fu_age_impute),
        ('ss',ss),
        ('knr',knr)
    ]

)
age_params = {
    
}
age_params2 = {'knr__n_neighbors':range(4,15)
    
}

In [18]:
gs_age_kn = GridSearchCV(age_impute_pipe_kn, param_grid=age_params2, cv=5)
gs_age_lr = GridSearchCV(age_impute_pipe_lr, param_grid=age_params, cv=5)

In [19]:
titanic_ages =titanic[(titanic['Age']!='?')]
titanic_no_ages = titanic[(titanic['Age']=='?')]

test_ages =testdf[(testdf['Age']!='?')]
test_no_ages = testdf[(testdf['Age']=='?')]


In [20]:
X_train = titanic_ages.drop(['Age','Survived'],axis=1)
y_train= titanic_ages['Age']


In [21]:
X_test1 =titanic_no_ages.drop(['Age','Survived'], axis=1)
X_test2 = test_no_ages.drop('Age', axis=1)

In [22]:
gs_age_lr.fit(X_train,y_train)
print(gs_age_lr.best_score_)

0.197285359444


In [23]:
gs_age_kn.fit(X_train,y_train)
print(gs_age_kn.best_score_)

0.237595317622


In [24]:
X_test1['Age'] = gs_age_kn.predict(X_test1)
X_test2['Age']= gs_age_kn.predict(X_test2)

In [25]:
titanic.loc[X_test1.index,'Age'] = X_test1['Age']
testdf.loc[X_test2.index,'Age'] = X_test2['Age']

In [26]:
def ageband(age):
    ageband= (int(age)//13)+1
    return ageband

In [27]:


titanic['AgeBand']=titanic.Age.apply(ageband)
testdf['AgeBand']=testdf.Age.apply(ageband)

In [28]:
def findsurname(wife):
    if wife[-1]==')':
        return wife.split()[0]
    
def findhusband(wife, husband):
    for num in range(len(wife.split())):
        if wife.split()[num][-1]=='.':
            try:
                return (wife.split()[num-1]==husband.split()[num-1])  & (wife.split()[num+1]==husband.split()[num+1])
            except:
                return False
            
        #Splitting by gender
def dead_husband(wife,husbandlist):
    #this takes a wife's name as an argument and a listas the second argument.  it returns a true or false
    result= False # assume that the wife did not die
    for num in range(len(husbandlist)):
        if findhusband(wife, husbandlist[num]):
            result = True
    return result
def dead_wife(husband,wifelist):
    result= False # assume that the wife did not die
    for num in range(len(wifelist)):
        if findhusband(wifelist[num], husband):
            result = True
    return result
            
titanicmale=  titanic[titanic['Sex']=='male'] 
titanicfemale= titanic[titanic['Sex']=='female']
testmale=  testdf[testdf['Sex']=='male'] 
testfemale= testdf[testdf['Sex']=='female']


In [29]:
livemen= titanicmale[titanicmale['Survived']==1]
deadmen= titanicmale[titanicmale['Survived']==0]
livewomen= titanicfemale[titanicfemale['Survived']==1]
deadwomen= titanicfemale[titanicfemale['Survived']==0]
deadwomennames= deadwomen.Name.values
livewomennames= livewomen.Name.values
livemennames = livemen.Name.values
deadmennames = deadmen.Name.values

In [30]:
# lets do women with dead husbands. 
titanicfemale['Dead_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
titanicfemale['Live_Husband']= titanicfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
titanicmale['Dead_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
titanicmale['Live_Wife']=titanicmale.Name.apply(lambda x:  dead_wife(x,livewomennames))
# lets do women with dead husbands. 
testfemale['Dead_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,deadmennames))
testfemale['Live_Husband']= testfemale.Name.apply(lambda x:  dead_husband(x,livemennames))
testmale['Dead_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,deadwomennames))
testmale['Live_Wife']=testmale.Name.apply(lambda x:  dead_wife(x,livewomennames))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [31]:
testdf.loc[testfemale.index,'Dead_Husband']= testfemale['Dead_Husband'].astype(int)

testdf.loc[testfemale.index,'Live_Husband']= testfemale['Live_Husband'].astype(int)
testdf.loc[testmale.index,'Dead_Wife']= testmale['Dead_Wife'].astype(int)
testdf.loc[testmale.index,'Live_Wife']= testmale['Live_Wife'].astype(int)
titanic.loc[titanicfemale.index,'Dead_Husband']= titanicfemale['Dead_Husband'].astype(int)
titanic.loc[titanicfemale.index,'Live_Husband']= titanicfemale['Live_Husband'].astype(int)
titanic.loc[titanicmale.index,'Dead_Wife']= titanicmale['Dead_Wife'].astype(int)
titanic.loc[titanicmale.index,'Live_Wife']= titanicmale['Live_Wife'].astype(int)
titanic['Dead_Husband'].fillna(value=0, inplace=True)
titanic['Dead_Wife'].fillna(value=0, inplace=True)
testdf['Dead_Husband'].fillna(value=0, inplace=True)
testdf['Dead_Wife'].fillna(value=0, inplace=True)
titanic['Live_Husband'].fillna(value=0, inplace=True)
titanic['Live_Wife'].fillna(value=0, inplace=True)
testdf['Live_Husband'].fillna(value=0, inplace=True)
testdf['Live_Wife'].fillna(value=0, inplace=True)

In [32]:
kmc = KMeans()

In [33]:
kmc.fit_predict(numeric_features(titanic))


array([4, 3, 4, 7, 4, 4, 7, 0, 4, 0, 4, 0, 4, 0, 4, 4, 0, 4, 4, 4, 0, 4, 4,
       0, 0, 0, 4, 5, 4, 4, 0, 1, 4, 4, 3, 7, 4, 4, 4, 4, 4, 0, 4, 7, 4, 4,
       4, 4, 0, 4, 0, 4, 3, 0, 7, 0, 4, 4, 0, 7, 4, 3, 3, 0, 0, 4, 4, 4, 4,
       4, 4, 7, 3, 4, 7, 4, 4, 4, 0, 4, 4, 4, 4, 7, 4, 4, 0, 4, 5, 4, 4, 4,
       7, 0, 4, 4, 0, 7, 0, 0, 4, 4, 3, 4, 4, 4, 4, 4, 4, 0, 7, 4, 4, 4, 4,
       4, 4, 0, 5, 0, 3, 4, 0, 4, 3, 4, 4, 4, 0, 4, 4, 4, 4, 0, 4, 4, 0, 7,
       4, 3, 4, 4, 4, 4, 4, 0, 4, 0, 0, 4, 4, 3, 4, 4, 4, 7, 4, 4, 4, 3, 4,
       4, 4, 4, 0, 0, 7, 0, 0, 7, 0, 0, 4, 4, 0, 4, 0, 0, 4, 4, 3, 4, 0, 0,
       0, 7, 4, 0, 4, 4, 4, 4, 4, 0, 0, 1, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4,
       0, 4, 0, 4, 0, 4, 4, 4, 6, 4, 0, 3, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 0,
       3, 4, 4, 0, 4, 4, 0, 0, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 7, 0, 4, 4, 0,
       4, 0, 4, 3, 3, 2, 0, 4, 0, 3, 4, 4, 4, 0, 4, 1, 1, 0, 4, 0, 0, 4, 3,
       4, 4, 0, 0, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 3, 3, 4, 4, 4, 0, 4, 1, 0,
       5, 4,

In [34]:
def dummies2(df):
    cols = [ 'Embarked','Deck', 'Title', 'Ticketnum']
    return pd.get_dummies(df[cols], columns=cols)
dummies_tf2 = FunctionTransformer(dummies2, validate=False)
def numeric_features2(df):
    return df[['Pclass', 'IsFemale',
       'AgeBand',  'Ticketnum']]
def kclass (df):
    df['Kclass'] =kmc.fit_predict(numeric_features(df))
    return df['Kclass']
kclass_tf= FunctionTransformer(kclass,validate=False)
numeric_features_tf2 = FunctionTransformer(numeric_features2, validate=False)

In [35]:
survive_transformers = [('numfeat' ,numeric_features_tf),('dummies' ,dummies_tf2)]

In [36]:
kclass_tf.fit_transform(titanic)

0      0
1      6
2      0
3      4
4      0
5      0
6      4
7      5
8      0
9      5
10     0
11     5
12     0
13     5
14     0
15     0
16     5
17     0
18     0
19     0
20     5
21     0
22     0
23     5
24     5
25     5
26     0
27     1
28     0
29     0
      ..
861    0
862    5
863    6
864    0
865    0
866    0
867    4
868    0
869    0
870    0
871    4
872    0
873    0
874    5
875    0
876    0
877    0
878    0
879    6
880    5
881    0
882    0
883    0
884    0
885    5
886    0
887    5
888    5
889    5
890    0
Name: Kclass, Length: 891, dtype: int32

In [37]:
prep_for_tensor=  FeatureUnion(survive_transformers)

In [38]:
pca = PCA(n_components=30)
knc = KNeighborsClassifier()

In [39]:
survive_pipe_kn =  Pipeline([('prep',prep_for_tensor),('ss',ss),('pca',pca)])


In [40]:
titanic.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Deck,Title,Ticketnum,cluster,AgeBand,Dead_Husband,Live_Husband,Dead_Wife,Live_Wife,Kclass
0,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,?,...,?,Mr.,2,0,2,0.0,0.0,0.0,0.0,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,...,C,Mrs.,1,5,3,0.0,0.0,0.0,0.0,6
2,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,?,...,?,Miss,3,0,3,0.0,0.0,0.0,0.0,0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,...,C,Mrs.,1,4,3,1.0,0.0,0.0,0.0,4
4,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,?,...,?,Mr.,3,0,3,0.0,0.0,0.0,0.0,0


In [41]:
titanic.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsMale',
       'IsAlone', 'Deck', 'Title', 'Ticketnum', 'cluster', 'AgeBand',
       'Dead_Husband', 'Live_Husband', 'Dead_Wife', 'Live_Wife', 'Kclass'],
      dtype='object')

In [42]:
X= survive_pipe_kn.fit_transform(titanic)
y= titanic['Survived']

In [43]:
X_test2 = survive_pipe_kn.fit_transform(testdf)

In [44]:
testdf.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare',
       'Cabin', 'Embarked', 'FamilyCount', 'IsFemale', 'IsMale', 'IsAlone',
       'Deck', 'Title', 'Ticketnum', 'cluster', 'AgeBand', 'Dead_Husband',
       'Live_Husband', 'Dead_Wife', 'Live_Wife'],
      dtype='object')

In [45]:
X_test2.shape

(418, 30)

In [46]:
kf = StratifiedKFold(random_state=2003)

In [47]:
cols = ['TN1','TN2','TN3']
shape = (titanic.shape[0], len(cols))
zeros= np.zeros(shape)
stacked_train = pd.DataFrame(zeros,columns = cols, index=titanic.index)

In [48]:
for train_ind,test_ind in kf.split(X,y):
    X_train=X[train_ind,:]
    X_test=X[test_ind,:]
    y_train=y[train_ind]
    y_test=y[test_ind]
    knc = KNeighborsClassifier()
    knc.fit(X_train,y_train)
    pred = knc.predict(X_test)
    stacked_train.loc[test_ind,'TN1'] = pred
    model = LogisticRegression()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    stacked_train.loc[test_ind, 'TN2'] = pred
    
    model = svm.SVC()
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    stacked_train.loc[test_ind, 'TN3'] = pred

    
    model = DecisionTreeClassifier(max_depth=1)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    stacked_train.loc[test_ind, 'TN4'] = pred

    

In [49]:
stacked_train.corr()

,TN1,TN2,TN3,TN4
TN1,1.000000,0.762545,0.813757,0.494973
TN2,0.762545,1.000000,0.918061,0.468042
TN3,0.813757,0.918061,1.000000,0.474878
TN4,0.494973,0.468042,0.474878,1.000000


In [50]:
stacked_modle = AdaBoostClassifier()
params = {}
gs = GridSearchCV(stacked_modle, param_grid=params)
gs.fit(stacked_train,y)


GridSearchCV(cv=None, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params={}, iid=True, n_jobs=1, param_grid={},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [51]:
gs.best_score_

0.97306397306397308

In [52]:
knc = KNeighborsClassifier()
lr = LogisticRegression()
svc = svm.SVC()
dtc = DecisionTreeClassifier(max_depth=1)
knc.fit(X,y)
lr.fit(X,y)
svc.fit(X,y)
dtc.fit(X,y)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [53]:
shape = (testdf.shape[0], 3)
zeros = np.zeros(shape)
stacked_test = pd.DataFrame(zeros, index=testdf.index, columns=cols)
stacked_test.head()

,TN1,TN2,TN3
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [54]:
testdf.shape

(418, 23)

In [55]:
stacked_test['TN2']= lr.predict(X_test2)

In [56]:
stacked_test['TN3']= svc.predict(X_test2)

In [57]:
stacked_test['TN1']= knc.predict(X_test2)

In [58]:
stacked_test['TN4']= dtc.predict(X_test2)

In [59]:
pred2= gs.predict(stacked_test)

In [60]:
pred2.shape

(418,)

In [61]:
testdf = pd.read_csv('test.csv')

In [62]:
testdf.shape

(418, 11)

In [63]:
submissionnew  = pd.DataFrame({'Survived':pred2}, index=testdf.PassengerId.values)

In [64]:
submissionnew.to_csv('skenkeltitanic.csv', index_label='PassengerId')

In [65]:
submissionnew.head()

,Survived
892,1
893,0
894,1
895,0
896,0
